# All try code

In [31]:
import mne
import numpy as np
from mne.datasets import eegbci
import matplotlib.pyplot as plt
from os import listdir
from mne.channels import make_standard_montage
from scipy import signal
from scipy.linalg import sqrtm, inv 

def GetRawEDF(target_subjects= "all", condition="offline"):

    EEG_data = {}

    if condition == "offline":
        condition = "Offline_Experiment"
    elif condition == "online":
        condition = "Online_Experiment"

    if target_subjects == "all":
        target_subjects = ["pipo","NutF8","AJpang","Aoomim","voen"]

    for i in range (0,len(target_subjects)):

        path = "C:\\git\Senior_Thesis\\DataSet\\"+condition+"\\"+ target_subjects[i] +"\\notch_EDF\\"
        list_dir = listdir(path)
        raw_each = [0] * len(list_dir)
        for j in range(len(list_dir)):
            raw_each[j] = mne.io.read_raw_edf(path+list_dir[j],preload = False)
            
        raw_edf = mne.concatenate_raws(raw_each)

        eegbci.standardize(raw_edf)  # set channel names
        montage = make_standard_montage("standard_1005")
        raw_edf.set_montage(montage)

        EEG_data[target_subjects[i]] = {"Raw_data": raw_edf.copy()}

    print(f"Successful to create Data of {target_subjects}")

    return EEG_data

EEG_data = GetRawEDF(target_subjects = "all")

Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Offline_Experiment\pipo\notch_EDF\sess1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Offline_Experiment\pipo\notch_EDF\sess2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Offline_Experiment\pipo\notch_EDF\sess3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Offline_Experiment\NutF8\notch_EDF\sess1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Offline_Experiment\NutF8\notch_EDF\sess2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\git\Senior_The

In [22]:
target_sub = ["pipo","nutF8","AJpang","Aoomim"]

a = [0,0,0,0]

EEG_data = {}

for i in target_sub:
    EEG_data[i] = {"Raw_data": a.copy()}

nor = [5,5,5]

for key in EEG_data:
    print(key)
    EEG_data[key]["EA_data"] = nor


# for sub_key in EEG_data:
#     for data_key in EEG_data[sub_key]:
#         print(sub_key,data_key)
    


pipo
nutF8
AJpang
Aoomim


In [19]:
EEG_data

{'pipo': {'data': [0, 0, 0, 0], 'EA_data': [5, 5, 5]},
 'nutF8': {'data': [0, 0, 0, 0], 'EA_data': [5, 5, 5]},
 'AJpang': {'data': [0, 0, 0, 0], 'EA_data': [5, 5, 5]},
 'Aoomim': {'data': [0, 0, 0, 0], 'EA_data': [5, 5, 5]}}

# Try CSP with multiple subjects

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit,StratifiedKFold ,cross_val_score, cross_val_predict, KFold

def computeCSPFeatures(data, target_subjects,condition):

    all_data = None
    label = None

    if condition == "noEA":
        query = "Raw_Epoch"
    else:
        query = "EA_Epoch"

    for sub in target_subjects:
        if all_data is None:
            all_data = data[sub]['Raw_Epoch']
        else:
            all_data = np.concatenate((all_data, data[sub][query]), axis=0)

        if label is None:
            label = data[sub]['label']
        else:
            label = np.concatenate((label, data[sub]['label']), axis=0)


    print(np.array(all_data).shape, np.array(label).shape)
    
    csp = CSP(n_components=8, reg=None, log=None, rank= 'info')

    train_data_csp, label = shuffle(all_data, label, random_state = 0)
    X_train, X_test, y_train, y_test = train_test_split(train_data_csp, label, test_size= 0.3, random_state = 0, stratify=label)
    csp.fit(X_train, y_train)

    X_train = csp.transform(X_train)
    X_test  = csp.transform(X_test)


    lda = LinearDiscriminantAnalysis()
    score = cross_val_score(lda, X_train, y_train, cv= 10)
    print("LDA only Cross-validation scores:", np.mean(score))
    lda.fit(X_train, y_train)
        
    from sklearn.metrics import classification_report,confusion_matrix

    y_pred = lda.predict(X_train)

    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true=y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_train, y_pred=y_pred))

    y_pred = lda.predict(X_test)

    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))

computeCSPFeatures(EEG_Epochs, target_subjects = ["pipo","voen","AJpang"] ,condition = "noEA")